In [ ]:
!pip install streamlit
!pip install transformers
!pip install SentencePiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow import squeeze
from transformers import XLNetTokenizer, TFXLNetModel
from sklearn.model_selection import train_test_split

DATA_PATH = 'https://raw.githubusercontent.com/Benrir/NLP/main/train_clickbait.csv'
data = pd.read_csv(DATA_PATH, encoding='utf-8')

X_xl = data['headline'].values
y_xl = data['clickbait'].values

X_train_xl, X_trainVal_xl, y_train_xl, y_trainVal_xl = train_test_split(X_xl, y_xl, test_size=0.2, random_state=0, stratify=y_xl)
X_test_xl, X_testVal_xl, y_test_xl, y_testVal_xl = train_test_split(X_trainVal_xl, y_trainVal_xl, test_size=0.5, random_state=0, stratify=y_trainVal_xl)


tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

def tokenize(data):
    input_ids = []
    for x in data:
        tokenized = tokenizer.encode_plus(
            x,
            add_special_tokens=True,
            max_length=128,
            padding='max_length',
            return_tensors='tf'
        )
        input_ids.append(tokenized['input_ids'])
    return np.concatenate(input_ids, axis=0)

X_train_xl = tokenize(X_train_xl)
X_test_xl = tokenize(X_test_xl)
X_testVal_xl = tokenize(X_testVal_xl)

word_inputs = Input(shape=(128,), name='word_inputs', dtype='int32')
xlnet = TFXLNetModel.from_pretrained('xlnet-base-cased')
xlnet_encodings = xlnet(word_inputs)[0]
doc_encoding = squeeze(xlnet_encodings[:, -1:, :], axis=1)
doc_encoding = Dropout(.1)(doc_encoding)
outputs = Dense(1, activation='sigmoid', name='outputs')(doc_encoding)
xlnetmodel = Model(inputs=[word_inputs], outputs=[outputs])
xlnetmodel.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=2e-5), metrics=['accuracy'])

epochs_xl = 1
history_xl = xlnetmodel.fit(X_train_xl, y_train_xl, validation_data=(X_testVal_xl, y_testVal_xl), epochs=epochs_xl, batch_size=16)

# Save the trained model
xlnetmodel.save('clickbait_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetModel: ['lm_loss']
- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLNetModel w

1600/1600 [==============================] - 914s 547ms/step - loss: 0.0621 - accuracy: 0.9788 - val_loss: 0.0105 - val_accuracy: 0.9953


In [ ]:
%%writefile app.py
import streamlit as st
import numpy as np
from tensorflow.keras.models import load_model
from transformers import XLNetTokenizer, TFXLNetModel

st.title("News Headline Clickbait Detection")
st.markdown("---")

headline = st.text_input("Enter the news headline")

if st.button("Detect"):
    # Load the trained model with custom objects
    model = load_model('clickbait_model.h5', custom_objects={'TFXLNetModel': TFXLNetModel})

    tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
    input_data = tokenizer.encode_plus(
        headline,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        return_tensors='tf'
    )['input_ids']

    input_data = np.reshape(input_data, (1, 128))  # Reshape input data

    prediction = model.predict(input_data)
    prediction_label = "Clickbait" if prediction[0][0] >= 0.5 else "Not Clickbait"

    st.markdown("### Prediction Result:")
    st.markdown(f"The input news headline is classified as **{prediction_label}**.")

Overwriting app.py


In [ ]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.624s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [ ]:
!streamlit run app.py &>/content/logs.txt & curl ipv4.icanhazip.com

35.227.37.68


In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 2.058s
^C
